# BASICS FROM THE DOMAIN

In [ ]:
# A simple solar panel
from IPython.core.display import Image, display
PATH = "C:/Users/Mehmet/Desktop/data/solar_power_ data/solar_panel.png"
display(Image(PATH, width=800, unconfined=True))

# CONVERTING CVS DATA TO DATAFRAME

In [ ]:
# Importing librraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Paths for the data
path0 = "/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv"
path1 = "/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv"
path2 = "/kaggle/input/solar-power-generation-data/Plant_2_Generation_Data.csv"
path3 = "/kaggle/input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv"
paths = [path0,path1,path2,path3]

In [ ]:
# Converting data to data frame
df0 = pd.read_csv(path0)
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

In [ ]:
# Summary of first data frame
df0

In [ ]:
# Summary of second data frame
df1

In [ ]:
# Summary of third data frame
df2

In [ ]:
# Summary of fourth data frame
df3

In [ ]:
# Solar plants
print("The data set include data for two solar plants which are {} and {}.".format(df0['PLANT_ID'].unique()[0],df2['PLANT_ID'].unique()[0]))
print('The plant {} has {} solar inverter.' .format(df0['PLANT_ID'].unique()[0],len( df0['SOURCE_KEY'].unique())))
print('The plant {} has {} solar inverter.'. format(df2['PLANT_ID'].unique()[0],len( df2['SOURCE_KEY'].unique())))     
  

In [ ]:
# Data frames' data types
for i in [df0,df1,df2,df3]:
    i.info()
    

In [ ]:
# The DATE_TIME columns attract the attention, they should be converted to date time
import dateutil.parser
df0.iloc[:,0] = df0.iloc[:,0].apply(lambda x:dateutil.parser.parse(x))
df1.iloc[:,0] = df1.iloc[:,0].apply(lambda x:dateutil.parser.parse(x))
df2.iloc[:,0] = df2.iloc[:,0].apply(lambda x:dateutil.parser.parse(x))
df3.iloc[:,0] = df3.iloc[:,0].apply(lambda x:dateutil.parser.parse(x))

In [ ]:
# Another issue comes from data frame type info, significant difference observation number difference among the data frames

a = pd.DataFrame(index=df0['DATE_TIME'].unique()).sort_index()
a['First Data Frame'] = 1
b = pd.DataFrame(index=df1['DATE_TIME'].unique()).sort_index()
b['Seconnd Data Frame'] = 1.01
c = pd.DataFrame(index=df2['DATE_TIME'].unique()).sort_index()
c['Seconnd Data Frame'] = 1.02
d = pd.DataFrame(index=df3['DATE_TIME'].unique()).sort_index()
d['Seconnd Data Frame'] = 1.03
print('As seen from the plot below, there is a significantce observation not overlaping the first data frame compare the other.')
print('The first data set is from {} to {}.'.format(a.index.min(),a.index.max()))
print('The second data set is from {} to {}.'.format(b.index.min(),b.index.max()))
print('The third data set is from {} to {}.'.format(c.index.min(),c.index.max()))
print('The third data set is from {} to {}.'.format(d.index.min(),d.index.max()))
fig = plt.figure(figsize=(10,8))
plt.plot(a,label='First Data Frame Observation Time Points')
plt.plot(b,label='Second Data Frame Observation Time Points',color='red',alpha=0.5)
plt.plot(c,label='Third Data Frame Observation Time Points',color='brown',alpha=0.5)
plt.plot(d,label='Fourth Data Frame Observation Time Points',color='black',alpha=0.5)
plt.title('Data Frames Observation Time Points')
plt.ylim(0.99, 1.04)
plt.legend(loc='best')
plt.show()


In [ ]:
# Data frames' missing values
for i in [df0,df1,df2,df3]:
    print(i.isna().any())

In [ ]:
# Data frames' descriptive statistics
for i in [df0,df1,df2,df3]:
    print(i.describe())

In [ ]:
# The first data frame's observation number is 68778, which covers entire year 2020. However, the time interval is 15 
# minutes; it means if there is no missing data, the data frame should have approximately 24*4*365 = 35040 data point for
# each inverter. For 22 inverter, that number reaches 770880 data points which is way far than the data size. Another way to
# looking at the missing observaation based on plants for the given period, which is following result:

df_2020 = pd.DataFrame(pd.date_range("2020-01-01", "2020-12-31", freq="15 Min"))
df_2020.columns = ['DATE_TIME']
missing_date_df0 =  pd.merge(df0,df_2020,how='right',on=['DATE_TIME'],indicator=True).reset_index(drop=True)
missing_date_df0[missing_date_df0['_merge']=='right_only']
print('There are {} missing observation for the year 2020 for the plant1 yield data set.'.format(len(missing_date_df0[missing_date_df0['_merge']=='right_only'])))

In [ ]:
# Merging yield data and temperature data
plant1 = pd.merge(df0,df1,how='left',on=['DATE_TIME','PLANT_ID']).reset_index()
plant2 = pd.merge(df2,df3,how='left',on=['DATE_TIME','PLANT_ID']).reset_index()

In [ ]:
# Adding hour, day, month and year columns to plant1's yield data set
plant1.set_index('DATE_TIME',inplace=True)

plant1['Minute'] = 0
plant1['Hour'] = 0
plant1['Day'] = 0
plant1['Month'] = 0
plant1['Year'] = 0
for i in plant1.index:
    
    plant1.loc[i,'Minute'] = i.minute
    plant1.loc[i,'Hour'] = i.hour
    plant1.loc[i,'Day'] = i.day
    plant1.loc[i,'Month'] = i.month
    plant1.loc[i,'Year'] = i.year

In [ ]:
# Adding hour, day, month and year columns to plant2's yield data set
plant2.set_index('DATE_TIME',inplace=True)

plant2['Minute'] = 0
plant2['Hour'] = 0
plant2['Day'] = 0
plant2['Month'] = 0
plant2['Year'] = 0
for i in plant2.index:
    
    plant2.loc[i,'Minute'] = i.minute
    plant2.loc[i,'Hour'] = i.hour
    plant2.loc[i,'Day'] = i.day
    plant2.loc[i,'Month'] = i.month
    plant2.loc[i,'Year'] = i.year

In [ ]:
#Plant1 & Plant2's Hourly Averaged  DC Power Production
fig = plt.figure(figsize=(10,8))
plant1.groupby(by='Hour').mean()['DC_POWER'].plot(kind='bar',label="Plant1's DC Power Production")
plant2.groupby(by='Hour').mean()['DC_POWER'].plot(kind='bar',label="Plant2's DC Power Production",color='red',alpha=0.5)
plt.title("Plant1 & Plant2's Hourly Averaged  DC Power Production")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant1 maximum hourly average DC_POWER and its hour
aa = plant1.groupby(by='Hour').mean()['DC_POWER'].reset_index()
aa[aa['DC_POWER']==aa.DC_POWER.max()].round(2)

In [ ]:
#Plant2 maximum hourly average DC_POWER and its hour
aa = plant2.groupby(by='Hour').mean()['DC_POWER'].reset_index()
aa[aa['DC_POWER']==aa.DC_POWER.max()].round(2)

In [ ]:
#P lant1 & Plant2's Hourly Averaged  AC Power Production
fig = plt.figure(figsize=(10,8))
plant1.groupby(by='Hour').mean()['AC_POWER'].plot(kind='bar',label="Plant1's AC Power Production")
plant2.groupby(by='Hour').mean()['AC_POWER'].plot(kind='bar',label="Plant2's AC Power Production",color='red',alpha=0.5)
plt.title("Plant1 & Plant2's Hourly Averaged  AC Power Production")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant1 maximum hourly average AC_POWER and its hour
aa = plant1.groupby(by='Hour').mean()['AC_POWER'].reset_index()
aa[aa['AC_POWER']==aa.AC_POWER.max()].round(2)

In [ ]:
#Plant2 maximum hourly average AC_POWER and its hour
aa = plant2.groupby(by='Hour').mean()['AC_POWER'].reset_index()
aa[aa['AC_POWER']==aa.AC_POWER.max()].round(2)

In [ ]:
#Plant1 Monthly Averaged  DC Power Production
fig = plt.figure(figsize=(10,8))
plant1.groupby(by='Month').mean()['DC_POWER'].plot(kind='bar',label="Plant1's DC Power Production")
plt.title("Plant1 Monthly Averaged  DC Power Production")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant2's Monthly Averaged  DC Power Production
fig = plt.figure(figsize=(10,8))
plant2.groupby(by='Month').mean()['DC_POWER'].plot(kind='bar',label="Plant2's DC Power Production",color='red',alpha=0.5)
plt.title("Plant2's Monthly Averaged  DC Power Production")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant1 maximum DC Power and its month
aa = plant1.groupby('Month').sum()[['DC_POWER']].reset_index()
aa[aa['DC_POWER']==aa.DC_POWER.max()].round(2)

In [ ]:
#Plant1 maximum AC Power and its month
aa = plant1.groupby('Month').sum()[['AC_POWER']].reset_index()
aa[aa['AC_POWER']==aa.AC_POWER.max()].round(2)

In [ ]:
#Plant2 maximum DC Power and its month
aa = plant2.groupby('Month').sum()[['DC_POWER']].reset_index()
aa[aa['DC_POWER']==aa.DC_POWER.max()].round(2)

In [ ]:
#Plant2 maximum AC Power and its month
aa = plant2.groupby('Month').sum()[['AC_POWER']].reset_index()
aa[aa['AC_POWER']==aa.AC_POWER.max()].round(2)

In [ ]:
#Plant1 minimum DC Power and its month
aa = plant1.groupby('Month').sum()[['DC_POWER']].reset_index()
aa[aa['DC_POWER']==aa.DC_POWER.min()].round(2)

In [ ]:
#Plant2 minimum DC Power and its month
aa = plant2.groupby('Month').sum()[['DC_POWER']].reset_index()
aa[aa['DC_POWER']==aa.DC_POWER.min()].round(2)

In [ ]:
#Plant1 minimum AC Power and its month
aa = plant1.groupby('Month').sum()[['AC_POWER']].reset_index()
aa[aa['AC_POWER']==aa.AC_POWER.min()].round(2)

In [ ]:
#Plant2 minimum AC Power and its month
aa = plant2.groupby('Month').sum()[['AC_POWER']].reset_index()
aa[aa['AC_POWER']==aa.AC_POWER.min()].round(2)

In [ ]:
# The plant1's total DC power production
round(plant1.groupby('Year').sum()[['DC_POWER']],2)

In [ ]:
# The plant2's total DC power production
round(plant2.groupby('Year').sum()[['DC_POWER']],2)

In [ ]:
# The plant1's total AC power production
round(plant1.groupby('Year').sum()[['AC_POWER']],2)

In [ ]:
# The plant2's total AC power production
round(plant2.groupby('Year').sum()[['AC_POWER']],2)

In [ ]:
#Plant1 & Plant2 Hourly Averaged Ambient Temperature
fig = plt.figure(figsize=(10,8))
plant1.groupby(by='Hour').mean()['AMBIENT_TEMPERATURE'].plot(kind='bar',label="Plant1 Hourly Averaged Ambient Temperature")
plant2.groupby(by='Hour').mean()['AMBIENT_TEMPERATURE'].plot(kind='bar',label="Plant2 Hourly Averaged Ambient Temperature",color='tomato',alpha=0.5)
plt.title("Plant1 & Plant2 Hourly Averaged Ambient Temperature")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant1 maximum hourly average Ambient Temperature and its hour
aa = plant1.groupby(by='Hour').mean()['AMBIENT_TEMPERATURE'].reset_index()
aa[aa['AMBIENT_TEMPERATURE']==aa.AMBIENT_TEMPERATURE.max()].round(2)

In [ ]:
#Plant2 maximum hourly average Ambient Temperature and its hour
aa = plant2.groupby(by='Hour').mean()['AMBIENT_TEMPERATURE'].reset_index()
aa[aa['AMBIENT_TEMPERATURE']==aa.AMBIENT_TEMPERATURE.max()].round(2)

In [ ]:
#Plant1 minimum hourly average Ambient Temperature and its hour
aa = plant1.groupby(by='Hour').mean()['AMBIENT_TEMPERATURE'].reset_index()
aa[aa['AMBIENT_TEMPERATURE']==aa.AMBIENT_TEMPERATURE.min()].round(2)

In [ ]:
#Plant2 minimum hourly average Ambient Temperature and its hour
aa = plant2.groupby(by='Hour').mean()['AMBIENT_TEMPERATURE'].reset_index()
aa[aa['AMBIENT_TEMPERATURE']==aa.AMBIENT_TEMPERATURE.min()].round(2)

In [ ]:
#Plant1 & Plant2 Hourly Averaged Irradiation
fig = plt.figure(figsize=(10,8))
plant1.groupby(by='Hour').mean()['IRRADIATION'].plot(kind='bar',label="Plant1 Hourly Averaged Irradiation")
plant2.groupby(by='Hour').mean()['IRRADIATION'].plot(kind='bar',label="Plant2 Hourly Averaged Irradiation",color='tomato',alpha=0.5)
plt.title("Plant1 & Plant2 Hourly Averaged Irradiation")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant1 maximum hourly average Irradiation and its hour
aa = plant1.groupby(by='Hour').mean()['IRRADIATION'].reset_index()
aa[aa['IRRADIATION']==aa.IRRADIATION.max()].round(2)

In [ ]:
#Plant2 maximum hourly average Irradiation and its hour
aa = plant2.groupby(by='Hour').mean()['IRRADIATION'].reset_index()
aa[aa['IRRADIATION']==aa.IRRADIATION.max()].round(2)

In [ ]:
#Plant1 minimum hourly average Irradiation and its hour
aa = plant1.groupby(by='Hour').mean()['IRRADIATION'].reset_index()
aa[aa['IRRADIATION']==aa.IRRADIATION.min()].round(2)

In [ ]:
#Plant2 minimum hourly average Irradiation and its hour
print("If there is no sunlight, there is no irradiation; however plant2's temrature measuring source reported irradiation\
 that indicates broken sensor.")
aa = plant2.groupby(by='Hour').mean()['IRRADIATION'].reset_index() 
aa[aa['IRRADIATION']==aa.IRRADIATION.min()].round(2)

In [ ]:
# Merged dta for plant1
plant1_new = pd.DataFrame(columns=list(plant1.columns)+['Yield'])

In [ ]:
# Merging plant1 data
for i in plant1['SOURCE_KEY_x'].unique():
    a = plant1[plant1['SOURCE_KEY_x']==i]
    a['Yield'] = a['DAILY_YIELD'].diff()
    plant1_new = plant1_new.append(a) 

In [ ]:
# Merged dta for plant1
plant2_new = pd.DataFrame(columns=list(plant2.columns)+['Yield'])

In [ ]:
# Merging plant2data
for i in plant2['SOURCE_KEY_x'].unique():
    a = plant2[plant2['SOURCE_KEY_x']==i]
    a['Yield'] = a['DAILY_YIELD'].diff()
    plant2_new = plant2_new.append(a)

In [ ]:
# Plant1 and plant2  missing yield values and negative values which are incorrect
plant1_new['Yield'] = plant1_new['Yield'].apply(lambda x: 0 if x<0 or x==np.nan else x) 
plant2_new['Yield'] = plant2_new['Yield'].apply(lambda x: 0 if x<0 or x==np.nan else x)

In [ ]:
# Plant1 monthly averaged yields
fig = plt.figure(figsize=(10,8))
plant1_new.groupby('Month').mean()['Yield'].plot(kind='bar',label="Plant1 Monthly Averaged Yields")
plt.title("Plant1 Monthly Averaged Yield")
plt.legend(loc='best')
plt.show()

In [ ]:
# Plant2 monthly averaged yields
fig = plt.figure(figsize=(10,8))
plant2_new.groupby('Month').mean()['Yield'].plot(kind='bar',label="Monthly Averaged Yields")
plt.title("Plant2 Monthly Averaged Yield")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant1 monthly yields
fig = plt.figure(figsize=(10,8))
plant1_new.groupby('Month').sum()['Yield'].plot(kind='bar',label="Monthly Yields")
plt.title("Plant1 Monthly Yields")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant2 monthly yields
fig = plt.figure(figsize=(10,8))
plant2_new.groupby('Month').sum()['Yield'].plot(kind='bar',label="Monthly Yields")
plt.title("Plant2 Monthly Yields")
plt.legend(loc='best')
plt.show()

In [ ]:
#Plant1 maximum yield and its month 
aa = plant1_new.groupby('Month').sum()[['Yield']].reset_index()
aa[aa['Yield']==aa.Yield.max()].round(2)

In [ ]:
#Plant2 maximum yield and its month 
aa = plant2_new.groupby('Month').sum()[['Yield']].reset_index()
aa[aa['Yield']==aa.Yield.max()].round(2)

In [ ]:
#Plant1 minumum yield and its month
aa = plant1_new.groupby('Month').sum()[['Yield']].reset_index()
aa[aa['Yield']==aa.Yield.min()].round(2)

In [ ]:
#Plant2 minumum yield and its month
aa = plant2_new.groupby('Month').sum()[['Yield']].reset_index()
aa[aa['Yield']==aa.Yield.min()].round(2)

In [ ]:
#Plant1 monthly average yield
plant1_new.groupby('Month').sum()[['Yield']].mean().round(2)

In [ ]:
#Plant2 monthly average yield
plant2_new.groupby('Month').sum()[['Yield']].mean().round(2)

In [ ]:
#Plant1 monthly median yield
plant1_new.groupby('Month').sum()[['Yield']].median().round(2)

In [ ]:
#Plant2 monthly median yield
plant2_new.groupby('Month').sum()[['Yield']].median().round(2)

In [ ]:
# Plant1 total yield in 2020
round(plant1_new.groupby('Year').sum()[['Yield']],2)

In [ ]:
# Plant2 total yield in 2020
round(plant2_new.groupby('Year').sum()[['Yield']],2)

In [ ]:
#Plant1 hourly averaged  maximum yield and its hour
aa = plant1_new.groupby('Hour').mean()[['Yield']].reset_index()
aa[aa['Yield']==aa.Yield.max()].round(2)

In [ ]:
#Plant2 hourly averaged  maximum yield and its hour
aa = plant2_new.groupby('Hour').mean()[['Yield']].reset_index()
aa[aa['Yield']==aa.Yield.max()].round(2)

In [ ]:
#Plant1 hourly averaged yield
plant1_new.groupby('Hour').sum()[['Yield']].mean().round(2)

In [ ]:
#Plant2 hourly averaged yield
plant2_new.groupby('Hour').sum()[['Yield']].mean().round(2)

In [ ]:
#Plant1 & Plant hourly averaged yields
fig = plt.figure(figsize=(10,8))
plant1_new.groupby('Hour').mean()['Yield'].plot(kind='bar',label="Plant1 Hourly Averaged Yields")
plant2_new.groupby('Hour').mean()['Yield'].plot(kind='bar',label="Plant2 Hourly Averaged Yields",color='tomato',alpha=0.5)
plt.title("Plant & Plant2 Hourly Averaged Yields")
plt.legend(loc='best')
plt.show()

In [ ]:
# Plant1 averaged hourly maximum yield and other parameters
aa = plant1_new.groupby('Hour').mean()[['DC_POWER','AC_POWER','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','Yield']].reset_index()
bb = aa[aa['Yield']==aa.Yield.max()].round(2)
bb

In [ ]:
# Plant2 averaged hourly maximum yield and other parameters
aa = plant2_new.groupby('Hour').mean()[['DC_POWER','AC_POWER','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','Yield']].reset_index()
bb = aa[aa['Yield']==aa.Yield.max()].round(2)
bb

In [ ]:
# Plant1's yield distribution among the inverters
plant1_inverters_yield = plant1_new.groupby(['Year','SOURCE_KEY_x']).sum().reset_index()[['SOURCE_KEY_x','Yield']].sort_values(by='Yield',ascending=False)
plant1_inverters_yield = plant1_inverters_yield.reset_index(drop=True)
plant1_inverters_yield.columns = ['Inverter','Yield']
plant1_inverters_yield

In [ ]:
# Plant1's inverters' yields

plant1_inverters_yield.plot(kind='bar',label="Inverters' Yields",figsize=(10,8))
plt.title("Plant1's Inverters' Yields")
plt.legend(loc='best')
plt.show()

In [ ]:
# Plant2's yield distribution among the inverters
plant2_inverters_yield = plant2_new.groupby(['Year','SOURCE_KEY_x']).sum().reset_index()[['SOURCE_KEY_x','Yield']].sort_values(by='Yield',ascending=False)
plant2_inverters_yield = plant2_inverters_yield.reset_index(drop=True)
plant2_inverters_yield.columns = ['Inverter','Yield']
plant2_inverters_yield

In [ ]:
# Plant2's inverters' yields

plant2_inverters_yield.plot(kind='bar',label="Inverters' Yields",figsize=(10,8))
plt.title("Plant2's Inverters' Yields")
plt.legend(loc='best')
plt.show()

In [ ]:
# Plant1's dc power distribution among the inverters
plant1_inverters_dc_power = plant1_new.groupby(['Year','SOURCE_KEY_x']).sum().reset_index()[['SOURCE_KEY_x','DC_POWER']].sort_values(by='DC_POWER',ascending=False)
plant1_inverters_dc_power = plant1_inverters_dc_power.reset_index(drop=True)
plant1_inverters_dc_power.columns = ['Inverter','DC_POWER']
plant1_inverters_dc_power

In [ ]:
# Plant1's inverters' dc powers

plant1_inverters_dc_power.plot(kind='bar',label="Inverters' DC Powers",figsize=(10,8))
plt.title("Plant1's Inverters' DC Powers")
plt.legend(loc='best')
plt.show()

In [ ]:
# Plant2's dc power distribution among the inverters
plant2_inverters_dc_power = plant2_new.groupby(['Year','SOURCE_KEY_x']).sum().reset_index()[['SOURCE_KEY_x','DC_POWER']].sort_values(by='DC_POWER',ascending=False)
plant2_inverters_dc_power = plant2_inverters_dc_power.reset_index(drop=True)
plant2_inverters_dc_power.columns = ['Inverter','DC_POWER']
plant2_inverters_dc_power

In [ ]:
# Plant2's inverters' dc powers

plant2_inverters_dc_power.plot(kind='bar',label="Inverters' DC Powers",figsize=(10,8))
plt.title("Plant2's Inverters' DC Powers")
plt.legend(loc='best')
plt.show()

In [ ]:
# Plant1's ac power distribution among the inverters
plant1_inverters_ac_power = plant1_new.groupby(['Year','SOURCE_KEY_x']).sum().reset_index()[['SOURCE_KEY_x','AC_POWER']].sort_values(by='AC_POWER',ascending=False)
plant1_inverters_ac_power = plant1_inverters_ac_power.reset_index(drop=True)
plant1_inverters_ac_power.columns = ['Inverter','AC_POWER']
plant1_inverters_ac_power

In [ ]:
# Plant1's inverters' dc powers
plant1_inverters_ac_power.plot(kind='bar',label="Inverters' AC Powers",figsize=(10,8))
plt.title("Plant1's Inverters' AC Powers")
plt.legend(loc='best')
plt.show()

In [ ]:
# Plant2's ac power distribution among the inverters
plant2_inverters_ac_power = plant2_new.groupby(['Year','SOURCE_KEY_x']).sum().reset_index()[['SOURCE_KEY_x','AC_POWER']].sort_values(by='AC_POWER',ascending=False)
plant2_inverters_ac_power = plant2_inverters_ac_power.reset_index(drop=True)
plant2_inverters_ac_power.columns = ['Inverter','AC_POWER']
plant2_inverters_ac_power

In [ ]:
# Plant2's inverters' dc powers

plant2_inverters_ac_power.plot(kind='bar',label="Inverters' AC Powers",figsize=(10,8))
plt.title("Plant2's Inverters' AC Powers")
plt.legend(loc='best')
plt.show()

# PREPARING DATA FOR PREDICTION

In [ ]:
#Preparing data for prediction, it is crutialto have data which doesn't have missing value for its corressponding day
# First We fix the data point which is nan consequense of diff operetion 
plant1_new.iloc[0,-1]=0
working_data = plant1_new.copy().sort_index()

In [ ]:
# a new column was generated with 1, by using groupby function for each day and inverter, days which have 96 (it indicates there is no missing data)
# selected for plant1 (selected data days have full data)
working_data.dropna(inplace=True)
working_data ['ones']=1
working_data0 = working_data.groupby(['Day','Month','Year','SOURCE_KEY_x']).sum()
working_data0 = working_data0[working_data0['ones']>=96]

In [ ]:
# a new column was generated with 1, by using groupby function for each day and inverter, days which have 96 (it indicates there is no missing data)
# selected for plant2 (selected data days have full data)
working_data1 = plant2_new.copy().sort_index()
working_data1.dropna(inplace=True)
working_data1['ones']=1
working_data2 = working_data1.groupby(['Day','Month','Year','SOURCE_KEY_x']).sum()
working_data2 = working_data2[working_data2['ones']>=96]

In [ ]:
# Selecting full day data pints for plant1
plant1_data = pd.DataFrame(columns=working_data.columns)
for i in range(len(working_data0)):
    a = working_data[(working_data['Day']==working_data0.index[i][0]) & (working_data['Month']==working_data0.index[i][1]) & (working_data['Year']==working_data0.index[i][2]) & (working_data['SOURCE_KEY_x']==working_data0.index[i][3])].copy()
    plant1_data = plant1_data.append(a)                                                                                                         
    

In [ ]:
 # Selecting full day data pints for plant1
plant2_data = pd.DataFrame(columns=working_data1.columns)
for i in range(len(working_data2)):
    a = working_data1[(working_data1['Day']==working_data2.index[i][0]) & (working_data1['Month']==working_data2.index[i][1]) & (working_data1['Year']==working_data2.index[i][2]) & (working_data1['SOURCE_KEY_x']==working_data2.index[i][3])].copy()
    plant2_data = plant2_data.append(a)    

In [ ]:
# Combining plant1 and plant2 data 
merged_plants_data = plant1_data.append(plant2_data)
merged_plants_data = merged_plants_data.reset_index(drop=False).drop(['Minute','Hour','Day','Month','Year'],axis=1)
merged_plants_data = merged_plants_data.groupby(['level_0']).sum().sort_index()

In [ ]:
merged_plants_data_hourly = merged_plants_data.resample('H').sum().sort_index()

In [ ]:
merged_plants_data_hourly

In [ ]:
# The exogonius variables must be shifted
merged_plants_data_hourly.iloc[:,:-1]=merged_plants_data_hourly.iloc[:,:-1].shift()
merged_plants_data_hourly.dropna(inplace=True)
df_hourly = merged_plants_data_hourly.copy()

In [ ]:
# TotaL yields over time
fig = plt.figure(figsize=(8,8))
df_hourly['Yield'].plot()
plt.show()

# CHECKING STATIONARY

In [ ]:
# Checking stationary, checking constant mean(We can see that the values are pretty close, it is still close
# enough for the time series to be stationary since we need to take into account the standard deviation.
#This method doesn’t prove or deny that our time series is stationary, it’s just indicates that it can be.)
cut = df_hourly['Yield'].index[int(0.5*len(df_hourly['Yield']))]
print('Mean before {}:'.format(cut))
print(round(df_hourly['Yield'].loc[:cut].mean(),2))
print('')
print('Mean after {}:'.format(cut))
print(round(df_hourly['Yield'].loc[cut:].mean(),2))
print('')

In [ ]:
# ADF test for stationary(Since p_value <0.05, we can reject the null hypothesis, we confirm our data is stationary. d=0)
from statsmodels.tsa.stattools import adfuller
result = adfuller(df_hourly['Yield'])
print('--------------------------')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

# DETERMINING SEASONILITY FREQUENCY

In [ ]:
# Determining seasonality frequency
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Finding seasonal frequency order(Clearly seasonal frequency is shown as 24)
fig, ax = plt.subplots(figsize=(8,8))
plot_acf(df_hourly['Yield'], lags=72, zero=False,ax=ax)
plt.show()

# NON-SEASONAL ORDERS

In [ ]:
# Finding non seasonal p,q orders (p=3,q=0)
# plot figure
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,8))
# Make ACF plot
plot_acf(df_hourly['Yield'].diff(24).dropna(), lags=24, zero=False, ax=ax1)
# Make PACF plot
plot_pacf(df_hourly['Yield'].diff(24).dropna(), lags=24, zero=False, ax=ax2)
plt.show()

# DECOMPOZING SEASONILITY

In [ ]:
# Import 
from statsmodels.tsa.seasonal import seasonal_decompose
# Decompose data
result = decomp_results = seasonal_decompose(df_hourly['Yield'], freq=24)

In [ ]:
# Plot decomposedn data(it looks stationay, D=0)

fig = result.plot()  
fig.set_size_inches(8, 12)

# DETERMINING SEASONAL ORDERS

In [ ]:
# Finding Seasonal Orders(P=2,Q=0)
# Create figure
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,8))
# Make ACF plot
plot_acf(df_hourly['Yield'], lags=[24,48,72,96], zero=False, ax=ax1)
# Make PACF plot
plot_pacf(df_hourly['Yield'],lags=[24,48,72,96], zero=False, ax=ax2)
plt.show()

# RUNING MODEL WITH THE DETERMINED ORDERS

In [ ]:
# Imports
from statsmodels.tsa.statespace.sarimax import SARIMAX
# Instantiate model
model = SARIMAX(df_hourly['Yield'],exog=df_hourly[['AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']], order=(3,0,0), seasonal_order=(2,0,0,24))
# Fit model
results = model.fit()

In [ ]:
#Summary
results.summary()

In [ ]:
# Make in-sample prediction
forecast = results.get_prediction(start=25)
# forecast mean
mean_forecast = forecast.predicted_mean
# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

In [ ]:
# Plot prediction
fig = plt.figure(figsize=(8,8))
plt.plot(mean_forecast,color='red',label='forecast')
plt.plot(df_hourly.iloc[-25:,-1],color='blue',label='actual')
plt.show()


In [ ]:

fig = results.plot_diagnostics()  
fig.set_size_inches(8, 12)

# DETERMINING ORDERS BY GRIDSEARCH (AIC)

In [ ]:
# Using AIC to find orders
import pmdarima as pm
results0 = pm.auto_arima(df_hourly['Yield'],exog=df_hourly[['AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']],
                        d=0,            # non-seasonal difference order
                        start_p=0,      # initial guess for p
                        start_q=0,      # initial guess for q
                        max_p=4,        # max value of p to test
                        max_q=4,        # max value of q to test
                        
                        seasonal=True,  # is the time series seasonal
                        m=24,            # the seasonal period
                        D=0,            # seasonal difference order
                        start_P=0,      # initial guess for P
                        start_Q=0,      # initial guess for Q
                        max_P=4,        # max value of P to test
                        max_Q=4,        # max value of Q to test
                        information_criterion='aic', # used to select best model
                        trace=True,             # print results whilst training
                        error_action='ignore',  # ignore orders that don't work
                        stepwise=True,          # apply intelligent order search
                        

)

In [ ]:
# Instantiate model
model_tuned = SARIMAX(df_hourly['Yield'],exog=df_hourly[['AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']], order=(3,0,0), seasonal_order=(4,0,1,24))
# Fit model
results_tuned = model.fit()
#Summary
results_tuned.summary()

In [ ]:
# Make in-sample prediction
forecast_tuned = results_tuned.get_prediction(start=25)
# forecast mean
mean_forecast_tuned = forecast_tuned.predicted_mean
# Get confidence intervals of forecasts
confidence_intervals_tuned = forecast_tuned.conf_int()

In [ ]:
# Plot prediction
fig = plt.figure(figsize=(8,8))
plt.plot(mean_forecast_tuned,color='red',label='forecast')
plt.plot(df_hourly.iloc[-25:,-1],color='blue',label='actual')
plt.show()

In [ ]:
  
fig = results.plot_diagnostics()  
fig.set_size_inches(8, 12)